### Student Number : 2
#### Alexandre Lemonnier (alexandre.lemonnier) (23077)
#### Victor Simonin (victor.simonin) (23093)

First, let's import our useful libraries

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import timedelta, datetime
from pyspark.ml.stat import Correlation
from sklearn.preprocessing import normalize
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from functools import reduce
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

In [3]:
spark_application_name = "Projet_LAVS"
spark = (SparkSession.builder.appName(spark_application_name).getOrCreate())

22/06/15 00:10:51 WARN Utils: Your hostname, leme-thinkpad resolves to a loopback address: 127.0.1.1; using 192.168.0.178 instead (on interface wlp4s0)
22/06/15 00:10:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/15 00:10:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Here we define the structure of the schema and its columns with their relative types

In [17]:
columns = [StructField("Date",TimestampType()), StructField("High",FloatType()), StructField("Low",FloatType()), StructField("Open",FloatType()), StructField("Close",FloatType()), StructField("Volume",FloatType()), StructField("Adj_Close",FloatType()), StructField("company_name",StringType())]
schema = StructType(columns)

def read_data(path):
    return spark.read.schema(schema).csv(path, header=True, sep=',')

Now let's read all the stocks' data :

In [18]:
amazon = 'data/AMAZON.csv'
apple = 'data/APPLE.csv'
facebook = 'data/FACEBOOK.csv'
google = 'data/GOOGLE.csv'
microsoft = 'data/MICROSOFT.csv'
tesla = 'data/TESLA.csv'
zoom = 'data/ZOOM.csv'

In [64]:
def predict_stock(stock_path):
    df_stock = read_data(stock_path).select('Date', 'Close', 'Volume')
    print("Stock columns:", df_stock.columns)
    print("Dataframe count:", df_stock.count())
    df_stock = df_stock.withColumn("row_id", monotonically_increasing_id())
    trainDF, testDF = df_stock.randomSplit([.8, .2], seed=42)
    
    logTrainDF = trainDF.withColumn("log_Close", log(col("Close")))
    logTestDF = testDF.withColumn("log_Close", log(col("Close")))
    
    print(f"There are {trainDF.cache().count()} rows in the training set, and {testDF.cache().count()} in the test set")
    
    rFormula = RFormula(formula="log_Close ~ . - Volume", featuresCol="features", labelCol="log_Close", handleInvalid="skip") 

    lr = LinearRegression(labelCol="log_Close", predictionCol="log_pred")

    pipeline = Pipeline(stages = [rFormula, lr])
    pipelineModel = pipeline.fit(logTrainDF)
    predDF = pipelineModel.transform(logTestDF)
    expDF = predDF.withColumn("prediction", exp(col("log_pred")))
    display(expDF.show())
    
    regressionEvaluator = RegressionEvaluator(labelCol="Close", predictionCol="prediction")
    rmse = regressionEvaluator.setMetricName("rmse").evaluate(expDF)
    r2 = regressionEvaluator.setMetricName("r2").evaluate(expDF)
    print(f"RMSE is {rmse}")
    print(f"R2 is {r2}")


In [65]:
predict_stock(google)

Stock columns: ['Date', 'Close', 'Volume']
Dataframe count: 987


22/06/15 00:49:15 WARN CacheManager: Asked to cache already cached data.
22/06/15 00:49:16 WARN CacheManager: Asked to cache already cached data.


There are 828 rows in the training set, and 159 in the test set


22/06/15 00:49:16 WARN Instrumentation: [f63eaffd] regParam is zero, which might cause numerical instability and overfitting.


+-------------------+------+---------+------+------------------+--------------------+------------------+-----------------+
|               Date| Close|   Volume|row_id|         log_Close|            features|          log_pred|       prediction|
+-------------------+------+---------+------+------------------+--------------------+------------------+-----------------+
|2017-01-05 00:00:00|794.02|1335200.0|     2| 6.677108674444709|[794.02001953125,...| 6.731700196030625|838.5717910555757|
|2017-01-11 00:00:00|807.91|1065900.0|     6| 6.694450632937915|[807.909973144531...| 6.742492187953101| 847.670660202503|
|2017-01-13 00:00:00|807.88|1099200.0|     8|  6.69441353868417|[807.880004882812...|  6.74259515767009|847.7579491044639|
|2017-01-23 00:00:00|819.31|1963600.0|    13| 6.708462519634765|[819.309997558593...| 6.751583232540753|855.4120071184622|
|2017-01-31 00:00:00|796.79|2160600.0|    19|  6.68059112841436|[796.789978027343...|6.7348707623333155|841.2347578442793|
|2017-02-06 00:0

None

RMSE is 23.698553069371386
R2 is 0.9879830930984685
